In [1]:
import numpy as np
import pandas as pd

In [2]:
d_f = pd.read_csv('Book-01.csv')

In [3]:
df = d_f[['Business Travel', 'CF_age band', 'CF_attrition label','Gender','Marital Status']]

In [4]:
df['CF_attrition label'] = df['CF_attrition label'].map({'Current Employees':'No','Ex-Employees':'Yes'})

C:\Users\Bilawal\AppData\Local\Temp\ipykernel_19460\3678304990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CF_attrition label'] = df['CF_attrition label'].map({'Current Employees':'No','Ex-Employees':'Yes'})


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
for col in df.columns:
    if df[col].dtype == 'object' or df[col].dtype.name == 'Category':
        df[col] = LabelEncoder().fit_transform(df[col])

C:\Users\Bilawal\AppData\Local\Temp\ipykernel_19460\2854025883.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = LabelEncoder().fit_transform(df[col])
C:\Users\Bilawal\AppData\Local\Temp\ipykernel_19460\2854025883.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = LabelEncoder().fit_transform(df[col])
C:\Users\Bilawal\AppData\Local\Temp\ipykernel_19460\2854025883.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [7]:
X = df.drop('CF_attrition label',axis=1)
y = df['CF_attrition label']

In [8]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [10]:
n_estimators = [int(x) for x in np.linspace(start=100, stop=1000, num=10)]

In [11]:
from sklearn.model_selection import RandomizedSearchCV

In [12]:
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]

In [13]:
min_samples_split = [2, 4, 10, 20, 50, 100]
min_samples_leaf = [1, 2, 3, 4, 5, 8, 10, 20, 50, 100, 200]

In [14]:
bootstrap = [True, False]

In [15]:
random_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [16]:
rf = RandomForestClassifier()

In [17]:
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=100,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

In [18]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 8,
                                                             10, 20, 50, 100,
                                                             200],
                                        'min_samples_split': [2, 4, 10, 20, 50,
                                                              100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, verbose=2)

In [19]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    accuracy = accuracy_score(test_labels, predictions)
    return accuracy

In [20]:
import time
start_time = time.time()
rf_random.fit(X_train, y_train)
end_time = time.time()
print('Elapsed time:', end_time - start_time)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Elapsed time: 42.00225329399109


In [21]:
base_model = RandomForestClassifier(n_estimators=5, random_state=42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

In [22]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

In [23]:
improvement = 100 * (random_accuracy - base_accuracy) / base_accuracy
print('Improvement: {:.2f}%'.format(improvement))

Improvement: -0.40%
